In [1]:
#Import Dependencies
import gmaps
import pandas as pd
import json
import folium

merge_data = pd.read_csv('data_sources/merge_data.csv')

ModuleNotFoundError: No module named 'folium'

In [ ]:
# create dataframe averaging all seven years of data for overall heatmap
# all_zips
all_zips = merge_data["Zip"].unique()
# get overall values for diversity index and property values
avg_div_index = merge_data.groupby(["Zip"])["Diversity Index"].mean()
avg_value = merge_data.groupby(["Zip"])["Value"].mean()
# Create dataframe
pd.options.display.float_format = '${:,.2f}'.format
ovrl_zips = pd.DataFrame({"Zip": all_zips, "Diversity Index": avg_div_index, "Value": avg_value})
ovrl_zips.head()

In [6]:
# create dataframe for each year
df_2011 = merge_data.loc[merge_data["Year"] == 2011]
df_2012 = merge_data.loc[merge_data["Year"] == 2012]
df_2013 = merge_data.loc[merge_data["Year"] == 2013]
df_2014 = merge_data.loc[merge_data["Year"] == 2014]
df_2015 = merge_data.loc[merge_data["Year"] == 2015]
df_2016 = merge_data.loc[merge_data["Year"] == 2016]
df_2017 = merge_data.loc[merge_data["Year"] == 2017]

In [2]:
# load geoJSON
with open("data_sources/zip_data/Austin_Zipcodes.geojson", "r") as json_file:
    data = json.load(json_file)
tmp = data
tmp

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'zipcode': '78702',
    'name': 'AUSTIN',
    'number_of_records': '45',
    'objectid': '51',
    'zipcodes_id': '45'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-97.691868384007, 30.248482149129],
       [-97.691727278076, 30.248110690045],
       [-97.69159318377, 30.247164548741],
       [-97.691420671847, 30.246030589478],
       [-97.691338740898, 30.245580589807],
       [-97.691876504586, 30.245434909015],
       [-97.692293577628, 30.245242731111],
       [-97.692438794752, 30.245174215551],
       [-97.692517136349, 30.244944800241],
       [-97.692678966586, 30.24476111083],
       [-97.693162756071, 30.244677570451],
       [-97.693744996442, 30.244642813074],
       [-97.694277111611, 30.244645525459],
       [-97.694561341234, 30.244706387897],
       [-97.694749349823, 30.244781769954],
       [-97.694995778683, 30.244863855677],
       [-97.695469792744, 30.24502213783

In [3]:
#get dataset of for all zip coded we will be using
zip_codes = [str(i) for i in merge_data["Zip"].unique()]
# zip_codes = race_df["Zip"].unique()
len(zip_codes)

NameError: name 'merge_data' is not defined

In [9]:
#remove zip codes not in our data set
zips = []
# no_zips = []
for i in range(len(tmp["features"])):
    if tmp["features"][i]["properties"]["zipcode"] in list(zip_codes):
        zips.append(tmp["features"][i])
len(zips)

67

In [10]:
# creating a new JSON object
new_json = dict.fromkeys(["type", "features"])
new_json["type"] = "FeatureCollection"
new_json["features"] = zips
new_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'zipcode': '78702',
    'name': 'AUSTIN',
    'number_of_records': '45',
    'objectid': '51',
    'zipcodes_id': '45'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-97.691868384007, 30.248482149129],
       [-97.691727278076, 30.248110690045],
       [-97.69159318377, 30.247164548741],
       [-97.691420671847, 30.246030589478],
       [-97.691338740898, 30.245580589807],
       [-97.691876504586, 30.245434909015],
       [-97.692293577628, 30.245242731111],
       [-97.692438794752, 30.245174215551],
       [-97.692517136349, 30.244944800241],
       [-97.692678966586, 30.24476111083],
       [-97.693162756071, 30.244677570451],
       [-97.693744996442, 30.244642813074],
       [-97.694277111611, 30.244645525459],
       [-97.694561341234, 30.244706387897],
       [-97.694749349823, 30.244781769954],
       [-97.694995778683, 30.244863855677],
       [-97.695469792744, 30.24502213783

In [11]:
#save JSON object as updated-file
with open("../zip_data/atx_zipcodes.json", "w") as outfile:
    json.dump(new_json, outfile, sort_keys=True, indent=4, separators=(',', ': '))

In [12]:
# Create function to generate maps
def create_map(table, zips, mapped_feature1, mapped_feature2, add_text = ''):
    # Reading of the updated GeoJSON file
    atx_geo1 = "../zip_data/atx_zipcodes.json"
    # Initiating a folium map with Austin's Longitude and Latitude
    m = folium.Map(location = [30.2672, -97.7431], zoom_start = 9)
    # creating a choropleth map for diversity
    m.choropleth(
        geo_data = atx_geo1,
        name = "Diversity Index",
        data = table,
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature1],
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.zipcode',
        fill_color = 'YlOrRd',
        fill_opacity = 0.7,
        line_opacity = 1.0,
    #     fill_color = branca.colormap.linear.YlOrRd_09.scale(0, 1),
        legend_name = (' ').join(mapped_feature1.split('_')).title() + ' by ' + add_text + ' in Austin'
    )
#     folium.LayerControl().add_to(m)
    
    # Create second layer
    # Reading of the updated GeoJSON file
    atx_geo2 = "../zip_data/atx_zipcodes.json"
    # creating a choropleth map for property values
    m.choropleth(
        geo_data = atx_geo2,
        name = "Property Value",
        data = table,
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature2],
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.zipcode',
        fill_color = 'PuBuGn',
        fill_opacity = 0.7,
        line_opacity = 1.0,
    #     fill_color = branca.colormap.linear.YlOrRd_09.scale(0, 1),
        legend_name = "Property" + (' ').join(mapped_feature2.split('_')).title() + ' by ' + add_text + ' in Austin'
    )
    folium.LayerControl().add_to(m)
    try:
        year = table["Year"].unique()[0].astype(str)
    except:
        year = "Overall"
        
    # save map with filename based on the feature of interest
    m.save(outfile = "../output_data/Combined " + year + "_" + mapped_feature1 + " and Property " + mapped_feature2 + '_map.html')


In [13]:
# Create map for all years combined
create_map(ovrl_zips, "Zip", "Diversity Index", "Value", "Zip Codes")

# Create map for each year
img_2011 = create_map(df_2011, "Zip", "Diversity Index", "Value", "Zip Codes")
create_map(df_2012, "Zip", "Diversity Index", "Value", "Zip Codes")
create_map(df_2013, "Zip", "Diversity Index", "Value", "Zip Codes")
create_map(df_2014, "Zip", "Diversity Index", "Value", "Zip Codes")
create_map(df_2015, "Zip", "Diversity Index", "Value", "Zip Codes")
create_map(df_2016, "Zip", "Diversity Index", "Value", "Zip Codes")
create_map(df_2017, "Zip", "Diversity Index", "Value", "Zip Codes")